In [14]:
import scipy.io
import pandas as pd

mat = scipy.io.loadmat("/disk/home/metabric/BRCA1View20000.mat")

In [16]:
gene_labels = [g[0] for g in mat['gene'][0]]
df = pd.DataFrame(mat['data'].transpose(), columns=gene_labels)
# df['id'] = mat['id'][0]
df['label'] = mat['targets']
df

,SCGB2A2,MUCL1,SCGB1D2,PIP,LOC648852,DNAJA2,TFF3,S100P,CPB1,CEACAM6,...,LOC641311,LOC645307,IL1RAP,LOC647149,LOC642453,LOC652100,LOC646050,LOC644912,LOC652294,label
0,6.365374,5.610561,5.134799,5.735802,5.855315,5.443985,8.005882,5.615179,6.682096,5.756946,...,5.409726,5.420998,5.396492,5.358751,5.424770,5.411055,5.242460,5.197767,5.643537,6
1,14.330442,7.979139,10.244240,7.138279,9.890661,11.195867,13.511164,10.548596,5.491578,6.805606,...,5.213895,5.167652,5.273045,5.287980,5.398061,5.226934,5.396744,5.352158,5.811813,4
2,14.388331,6.016715,12.164435,5.272691,6.468386,5.265518,13.386034,8.764722,7.331663,11.967719,...,5.707537,5.327874,5.549589,5.333051,5.368544,5.296182,5.694022,5.459744,5.269015,4
3,13.111243,7.251862,8.389121,8.664654,10.378585,5.544646,11.159766,8.263224,5.497906,5.923506,...,5.351022,5.009093,5.345468,5.176322,5.326991,5.480259,5.292691,5.242160,5.635111,5
4,12.243103,7.293489,10.919581,9.566137,8.503568,11.492583,11.531352,9.818938,9.443134,6.889670,...,5.320883,5.487783,5.479967,5.332074,5.273141,5.282872,5.222482,5.186136,5.616094,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,7.199176,5.708891,5.672914,14.464282,9.816897,5.604835,13.303832,6.391817,13.010996,8.294409,...,5.695733,5.363557,5.394837,5.442947,5.423763,5.558225,5.575000,5.636068,5.454823,3
2129,10.354497,14.464282,7.633244,14.464282,11.294018,5.468214,10.886420,12.260033,5.662372,6.274380,...,5.537254,5.613097,5.254552,5.205904,5.486546,5.206480,5.161718,5.451296,5.215587,5
2130,9.294401,13.703049,6.063617,10.063176,5.393440,5.529584,10.937517,12.220400,9.985646,8.370227,...,5.520596,5.572326,5.317225,5.178158,5.316757,5.210332,5.239428,5.395402,5.501265,2
2131,14.464282,13.203708,14.169804,13.731721,12.478907,5.347654,14.464282,9.593148,6.165212,11.207149,...,5.650387,5.359733,5.374522,5.245155,5.453742,5.353135,5.235716,5.155543,5.442698,5


In [17]:
import scipy.io
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# create dataloader
class BaselineMetabricDataset(Dataset):
    def __init__(self, mat_file):
        mat = scipy.io.loadmat(mat_file)
        gene_labels = [g[0] for g in mat['gene'][0]]
        self.df = pd.DataFrame(mat['data'].transpose(), columns=gene_labels)
        # df['id'] = mat['id'][0]
        self.df['label'] = mat['targets']
        self.df['label'] = self.df['label'] - 1
        self.labels = self.df.pop('label')
        self.num_classes = 6
        
        # maybe normalize the columns
        self.tensors = {}
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if idx in self.tensors:
            return self.tensors[idx]
        label = torch.zeros(self.num_classes)
        label[self.labels[idx]] = 1
        sample = (torch.FloatTensor(self.df.iloc[idx, :].values), label)
        self.tensors[idx] = sample
        return sample

In [18]:
mat_file = "/disk/home/metabric/BRCA1View20000.mat"
met_dataset = BaselineMetabricDataset(mat_file)

In [19]:
len(met_dataset)

2133

In [20]:
train_split, test_split = torch.utils.data.random_split(met_dataset, [1900, 233])
train_loader = DataLoader(train_split, batch_size=64, shuffle=True)
test_loader = DataLoader(test_split, batch_size=64, shuffle=True)

In [21]:
print(len(train_loader))
print(len(test_loader))

30
4


In [36]:
IN_DIM = 20000
OUT_DIM = 6
LEARNING_RATE = 1e-5
N_EPOCHS = 300
BATCH_SIZE = 100

model = nn.Sequential(
    nn.Linear(IN_DIM, 1024),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, OUT_DIM)
)

# loss_fn = nn.MSELoss(reduction='sum')
# loss_fun = nn.SmoothL1Loss()
loss_fun = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(N_EPOCHS):
    loss = None
    for i, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
    print(epoch, loss.item())

0 31.690187454223633
1 33.68888854980469
2 29.41666030883789
3 29.881725311279297
4 30.074764251708984
5 32.36648941040039
6 30.185632705688477
7 30.240121841430664
8 26.700206756591797
9 25.371135711669922
10 25.15553855895996
11 25.775842666625977
12 26.27074432373047
13 24.06377410888672
14 29.61651039123535
15 23.221153259277344
16 23.82718276977539
17 21.352779388427734
18 23.250396728515625
19 22.20803451538086
20 24.161170959472656
21 20.347013473510742
22 23.67109489440918
23 20.703014373779297
24 21.95526123046875
25 20.583215713500977
26 21.137752532958984
27 22.881820678710938
28 23.679471969604492
29 21.250619888305664
30 18.56313133239746
31 21.149890899658203
32 18.698150634765625
33 23.593774795532227
34 22.067312240600586
35 20.692705154418945
36 19.077468872070312
37 19.12200355529785
38 22.232967376708984
39 17.669448852539062
40 16.11856460571289
41 20.074800491333008
42 21.56955337524414
43 24.01909637451172
44 19.177433013916016
45 16.722436904907227
46 18.13866233

In [37]:
from sklearn.metrics import classification_report

correct = 0
total = 0

y_true = []
predicted = []

with torch.no_grad():
    for X, y in test_loader:
        predictions = model(X)
        _, pred = torch.max(predictions.data, 1)
        _, labeled = torch.max(y.data, 1)
        total += y.size(0)
        correct += (pred == labeled).sum().item()
        y_true.extend(labeled)
        predicted.extend(pred)
        print(pred)
        print(labeled)
        print("---------------------------------")
print(f"Accuracy: {correct} / {total} {100 * correct / total:.2f}%")


print(classification_report(y_true, predicted))

tensor([5, 0, 2, 2, 2, 3, 2, 1, 4, 3, 0, 3, 1, 5, 3, 2, 4, 2, 2, 2, 3, 2, 4, 2,
        2, 4, 2, 2, 0, 3, 3, 3, 3, 2, 2, 0, 4, 3, 1, 4, 2, 3, 3, 3, 2, 2, 2, 3,
        2, 0, 0, 4, 2, 2, 3, 0, 3, 2, 2, 0, 3, 2, 2, 2])
tensor([5, 0, 2, 2, 2, 3, 2, 1, 4, 2, 0, 3, 1, 5, 3, 1, 4, 2, 2, 4, 3, 2, 4, 2,
        2, 4, 2, 2, 0, 3, 3, 3, 3, 3, 2, 0, 0, 3, 0, 2, 2, 3, 3, 3, 2, 2, 2, 3,
        5, 1, 0, 4, 2, 3, 1, 0, 3, 2, 2, 0, 3, 2, 4, 2])
---------------------------------
tensor([1, 2, 3, 1, 3, 0, 1, 3, 2, 5, 2, 2, 2, 2, 0, 2, 3, 3, 2, 5, 5, 2, 2, 2,
        2, 5, 2, 4, 4, 1, 2, 5, 2, 2, 0, 2, 3, 2, 0, 2, 3, 2, 3, 4, 3, 3, 2, 2,
        2, 1, 3, 2, 2, 3, 3, 0, 1, 2, 3, 2, 2, 3, 3, 3])
tensor([0, 2, 3, 1, 3, 0, 1, 3, 3, 5, 2, 2, 3, 2, 0, 3, 3, 3, 2, 5, 5, 2, 2, 2,
        2, 5, 2, 4, 4, 1, 2, 5, 2, 2, 0, 2, 3, 2, 0, 2, 3, 2, 3, 4, 3, 3, 2, 2,
        2, 0, 3, 2, 2, 3, 3, 0, 3, 2, 3, 2, 2, 3, 3, 3])
---------------------------------
tensor([3, 2, 2, 2, 3, 3, 3, 2, 2, 2, 3, 0, 2, 2, 3, 2, 1, 2, 2,

In [38]:
torch.save(model.state_dict(), 'baseline_model_v0.4.pth')